In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from pathlib import Path

import torch

from transformers import AutoModelForCausalLM
from peft import PeftModel

import sys
sys.path.insert(0, str("/fsx/home-augustas/mlmi-thesis/src/ppo"))
from utils import get_tokenizer

In [3]:
device = torch.device("cuda:2")
device

device(type='cuda', index=2)

In [29]:
tokenizer = get_tokenizer("huggyllama/llama-7b")
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token
tokenizer

Loading tokenizer huggyllama/llama-7b...


Using pad_token, but it is not set yet.


Loaded tokenizer.



LlamaTokenizerFast(name_or_path='huggyllama/llama-7b', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)

In [31]:
tokenizer_2 = AutoTokenizer.from_pretrained("AugustasM/vicuna_rlhfed_v1")
tokenizer_2

LlamaTokenizerFast(name_or_path='AugustasM/vicuna_rlhfed_v1', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "lmsys/vicuna-7b-v1.3",
    low_cpu_mem_usage=True,
    device_map={ "": device },
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [7]:
peft_weight_path = Path("/fsx/home-augustas/ppo_logs/vicuna_UQA_3b_qnli_20230803_144559_52437/checkpoints/model_step_1_32")

model = PeftModel.from_pretrained(model, peft_weight_path)
model = model.merge_and_unload()

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [9]:
# Push to hub
output_name = "vicuna_rlhfed_v1"

model.save_pretrained(output_name)
tokenizer.save_pretrained(output_name)

model.push_to_hub(output_name, private=True)
tokenizer.push_to_hub(output_name, private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AugustasM/vicuna_rlhfed_v1/commit/92c9f2342c1d9df11d1b4494af9333f177f651ff', commit_message='Upload tokenizer', commit_description='', oid='92c9f2342c1d9df11d1b4494af9333f177f651ff', pr_url=None, pr_revision=None, pr_num=None)